In [ ]:

import numpy as np
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
import seaborn as sb
import os
import matplotlib.dates as mdates
import seaborn as snb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import glob
from wordcloud import WordCloud
%matplotlib inline

In [ ]:
path =  "/kaggle/input/learnplatform-covid19-impact-on-digital-learning/engagement_data"
last_csv = "new_combined_csv.csv"

def get_csv_file(path, last_csv):
    os.chdir(path)
    extension = 'csv'
    all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
    li = []
    for f in all_filenames:
#         print(f)
        df = pd.read_csv(f)
        district_id = f.split(".")[0]
        df["district_id"] = district_id
        li.append(df)
#combine all files in the list
    combined_csv = pd.concat(li)
    combined_csv = combined_csv.reset_index(drop=True)
    return combined_csv

df_tmp = get_csv_file(path, last_csv)

In [ ]:
df_tmp

In [ ]:
# checking for duplicates
df_tmp.duplicated().sum()

In [ ]:
# checking for null values
df_tmp.isnull().sum()

In [ ]:
df_tmp.describe()

In [ ]:
df_tmp['engagement_index'] = df_tmp['engagement_index'].fillna(0)
df_tmp['pct_access'] = df_tmp['pct_access'].fillna(0)
df_tmp.isnull().sum()

In [ ]:
df_tmp.dropna(inplace=True)
df_tmp.isnull().sum()

In [ ]:
df_tmp.info()

In [ ]:
df_tmp['lp_id'] = df_tmp['lp_id'].astype(int).astype(str)

In [ ]:
df_tmp.info()

In [ ]:
df_tmp['lp_id'].value_counts()

In [ ]:
df_tmp['time'].value_counts()

In [ ]:
lp_id_count_time = df_tmp.groupby(['lp_id', 'time']).size().reset_index(name='lp_id_count')
lp_id_count_time = lp_id_count_time.sort_values(["lp_id_count"], ascending=False)
lp_id_count_time

In [ ]:
df2 = df_tmp.copy()

In [ ]:
def get_day_month_year (df):
    df['time'] = pd.to_datetime(df['time'])
    df['year'] = (df.time.dt.strftime('%Y')).astype('int')
    df['day'] = df.time.dt.day
    df['month'] = df.time.dt.strftime('%B')
    return df
get_day_month_year(df2)

In [ ]:
df2['month'].value_counts()

In [ ]:
lp_id_count_mnth = df2.groupby(['lp_id', 'month']).size().reset_index(name='count')
lp_id_count_mnth = lp_id_count_mnth.sort_values(["count"], ascending=False)
lp_id_count_mnth

In [ ]:
# df2['district_id'] = df2['district_id'].apply(lambda x: str(x))

In [ ]:
month_lst = list(set(df2['month']))

In [ ]:
for i in month_lst:
    print(i)

In [ ]:
for i in month_lst:
    tmp = df2[df2['month'] == i]
    tmp = tmp.sort_values(by=['engagement_index'],ascending = False)[:20]
    sns.displot(tmp, x=tmp['engagement_index'], hue=tmp['district_id'], kind="kde")
    plt.xlabel("District id")
    plt.ylabel("count")
    plt.title(f"Highest Engagement index in {i}")
plt.show();

In [ ]:
plt.figure(figsize=[20, 15]);
df2['month'].value_counts().plot(x="month", y="count", kind="bar", rot=50, fontsize=35);

In [ ]:
df2_aggregated = df2.groupby(['time','month','day']).agg({'engagement_index':np.mean,'pct_access':np.mean})
df2_aggregated = df2_aggregated.reset_index()
df2_aggregated

In [ ]:
plt.figure(figsize=(15,11))
sns.lineplot(y=df2_aggregated['engagement_index'],x=df2_aggregated["month"],palette='rocket')
plt.title("Average engagement_index each month");

In [ ]:
plt.figure(figsize=(15,11))
sns.lineplot(y=df2_aggregated['pct_access'],x=df2_aggregated["month"],palette='rocket')
plt.title("Average pct_access each month");

In [ ]:
plt.figure(figsize=(15,10));
sns.lineplot(y=df2_aggregated['engagement_index'],x=df2_aggregated["day"]);
plt.title("Average engagement_index each day");

In [ ]:
df_product = pd.read_csv('/kaggle/input/learnplatform-covid19-impact-on-digital-learning/products_info.csv')
df_product

In [ ]:
df_product.isnull().sum()

In [ ]:
df_product.dropna(inplace=True)

In [ ]:
df_product.isnull().sum()

In [ ]:
df_product.rename(columns={'LP ID':'lp_id'}, inplace=True)

In [ ]:
df_product['lp_id'] = df_product['lp_id'].astype(int).astype(str)

In [ ]:
df_product['Provider/Company Name'].value_counts()

In [ ]:
df_product['Sector(s)'].value_counts()

In [ ]:
df_product.info()

In [ ]:
primary_essential_abbv = []
Primary_Essential_Function = []
for i in df_product['Primary Essential Function']:
    primary_essential_abbv.append(i.split('-',1)[0].strip())
    Primary_Essential_Function.append(i.split('-',1)[1].strip())
df_product['primary_essential_abbv']  = primary_essential_abbv
df_product['Primary_Essential_Function'] = Primary_Essential_Function
df_product

In [ ]:
df_product['Primary_Essential_Function'].value_counts()

In [ ]:
df_product['Primary_Essential_Function'] = df_product['Primary_Essential_Function'].str.replace(",","").str.replace("-","").str.replace("&","").str.replace("(","").str.replace(")","")